# Deploy Charmed Kubeflow on Charmed Kubernetes cluster on AWS

Contents:

## Prerequisites
- `jq`
- `kubectl`
- `juju`


## Setup AWS cloud

Follow [this guide](https://juju.is/docs/olm/amazon-aws) to setup AWS cloud and Juju controller.

## Deploy Charmed Kubernetes cluster on AWS

Checkout Charmed Kubernetes bundle release 1.24 and update cpu, disk, and memory constraints to satisfy Kubeflow requirements:

In [ ]:
git clone https://github.com/charmed-kubernetes/bundle.git
sed -i '/^ *charm: kubernetes-worker/,/^ *[^:]*:/s/constraints: cores=2 mem=8G root-disk=16G/constraints: cores=8 mem=32G root-disk=100G/' ./bundle/releases/1.24/bundle.yaml

`bundle.yaml` should contain updated constraints for `kubernetes-worker`:

```shell
  kubernetes-worker:
    annotations:
      gui-x: '90'
      gui-y: '850'
    channel: 1.24/stable
    charm: kubernetes-worker
    constraints: cores=8 mem=32G root-disk=100G
    expose: true
    num_units: 3
    options:
      channel: 1.24/stable
```

Deploy updated Charmed Kubernetes bundle on AWS with storage overlay:

In [ ]:
juju deploy ./bundle/releases/1.24/bundle.yaml --overlay ./bundle/overlays/aws-storage-overlay.yaml --trust
juju scp kubernetes-control-plane/0:config ~/.kube/config
juju add-k8s charmed-k8s-aws --controller $(juju switch | cut -d: -f1) --storage=cdk-ebs

## Deploy Charmed Kubeflow on created Charmed Kubernetes cluster on AWS

Add `kubeflow` model to the controller and deploy desired version of Charmed Kubeflow, eg. version 1.7:

In [ ]:
juju add-model kubeflow charmed-k8s-aws
juju deploy kubeflow --channel 1.7/stable --trust

Change gateway service type to `Nodeport`:

In [ ]:
kubectl -n kubeflow patch svc istio-ingressgateway-workload -p '{"spec":{"type":"NodePort"}}'

The updated `istio-ingressgateway-workload` service should have type updated to `NodePort`:

```shell
apiVersion: v1
kind: Service
metadata:
. . .
  name: istio-ingressgateway-workload
  namespace: kubeflow
spec:
  clusterIP: 10.152.183.219
  clusterIPs:
  - 10.152.183.219
  externalTrafficPolicy: Cluster
  internalTrafficPolicy: Cluster
  ipFamilies:
  - IPv4
  ipFamilyPolicy: SingleStack
  ports:
  - name: http2
    nodePort: 31670
    port: 80
    protocol: TCP
    targetPort: 8080
  - name: https
    nodePort: 32359
    port: 443
    protocol: TCP
    targetPort: 8443
  selector:
    istio: ingressgateway
  sessionAffinity: None
  type: NodePort
status:
  loadBalancer: {}
```

Set the `dex-auth` and `oidc-gatekeeper` public URLs with IP address of the node and the port value equal to `NodePort` of `istio-ingressgateway-workload` service:

In [ ]:
NODE_IP=$(kubectl -n kubeflow get nodes -o wide | sed -n '2 p' | awk '{print $7}')
NODE_PORT=$(kubectl -n kubeflow get svc istio-ingressgateway-workload -o=json | jq '(.spec.ports) | .[] | select(.name=="http2") | (.nodePort)')
juju config dex-auth public-url="http://$NODE_IP:$NODE_PORT"
juju config oidc-gatekeeper public-url="http://$NODE_IP:$NODE_PORT"
curl -f -LI "http://$NODE_IP:$NODE_PORT"